In [28]:
import numpy as np
import scipy.special as scp

In [29]:
# neural network class definition
class NeuralNetwork:

    # initialize the neural network
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # set number of nodes in input, hidden and output layer
        self.inodes = input_nodes
        self.hnodes = hidden_nodes
        self.onodes = output_nodes

        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w21 w22 etc
        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))

        # set the learning rate
        self.lrate = learning_rate

        # activation function is the sigmoid function
        self.activation_function = lambda x: scp.expit(x)

        pass

    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T

        # calculate signals into hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)

        # calculate signals into final output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)

        # output layer error (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden layer nodes
        hidden_errors = np.dot(self.who.T, output_errors)

        # update the weights for the links between the hidden and output layers
        self.who += self.lrate * np.dot((output_errors * final_outputs * (1.0 - final_outputs)),
                                        np.transpose(hidden_outputs))
        # update the weights for the links between the input and hidden layers
        self.wih += self.lrate * np.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)),
                                        np.transpose(inputs))

        pass

    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = np.array(inputs_list, ndmin=2).T

        # calculate signals into hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)

        # calculate signals into final output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)

        return final_outputs

In [30]:
# number of input, hidden and output layer
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

# set the learning ratio
learning_rate = 0.2

# create an instance of neural network
n = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [31]:
# load the mnist training data
data_file = open("mnist_train.csv")
training_data_list = data_file.readlines()
data_file.close()

In [32]:
# training of neural network
# epochs is the number of times the training data set is used for training
epochs = 1

for i in range(epochs):
    # go through all the records in dataset
    for record in training_data_list:
        # split the record by commas
        all_values = record.split(',')

        # scale and shift the inputs
        inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01

        # create the target output values
        targets = np.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99

        # train the neural network
        n.train(inputs, targets)
        pass
    pass

In [33]:
# load the mnist test data
data_file = open("mnist_test.csv")
test_data_list = data_file.readlines()
data_file.close()

In [34]:
# test the neural network

# scorecard to determine how well the neural network performs
scorecard = []

# go through all the records in dataset
for record in test_data_list:
    # split the record by commas
    all_values = record.split(',')

    # list of correct answers
    correct_label = int(all_values[0])

    # scale and shift the inputs
    inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01

    # query the network
    outputs = n.query(inputs)

    # the index of the highest value corresponds to the label
    label = np.argmax(outputs)

    # determine if correct or not
    if (label == correct_label):
        # network's answers is correct, thus add 1 to sscorecard
        scorecard.append(1)
    else:
        # network's answers is not correct, thus add 0 to sscorecard
        scorecard.append(0)
        pass
    pass

In [35]:
# calculate the performance score, the fraction of correct answers
scorecard_array = np.asarray(scorecard)
print("performance = ", scorecard_array.sum() / scorecard_array.size)

performance =  0.9583
